In [33]:
import numpy as np
import pandas as pd
import ast
import stanza
from tqdm import tqdm

In [34]:
bad_words = pd.read_csv('data/bad_word.txt',header=None)
bad_words_set = set(bad_words)

In [35]:
dataset = pd.read_csv('data/news_comments.csv')

In [36]:
cleaned_comments = pd.read_csv('data/comments_preprocessed.csv')

In [37]:
len(cleaned_comments)

63374

In [38]:
#Truncate the comments to 20 words
for i in tqdm(range(len(cleaned_comments))):
    cleaned_comments['comment'][i] = ' '.join(cleaned_comments['comment'][i].split()[:20])
    

  0%|          | 0/63374 [00:00<?, ?it/s]/tmp/ipykernel_71509/2735669482.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_comments['comment'][i] = ' '.join(cleaned_comments['comment'][i].split()[:20])
100%|██████████| 63374/63374 [00:16<00:00, 3850.63it/s]


In [39]:
dataset['cleaned_comment'] = cleaned_comments['comment']

In [40]:
pos_tagger = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos',use_gpu=True)

2023-04-24 14:22:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-04-24 14:22:26 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-04-24 14:22:26 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

2023-04-24 14:22:26 WARNING: GPU requested, but is not available!
2023-04-24 14:22:26 INFO: Using device: cpu
2023-04-24 14:22:26 INFO: Loading: tokenize
2023-04-24 14:22:26 INFO: Loading: pos
2023-04-24 14:22:27 INFO: Done loading processors!


In [47]:
dataset.tail()

,index,comment,cleaned_comment
65689,65689,They already have enough local staff to fill t...,NaN
65690,65690,Indeed it is! It still is ridiculously below t...,NaN
65691,65691,Amazon did because it's their play for DoD con...,NaN
65692,65692,"Thanks, I didn't get past the paywall.",NaN
65693,65693,"This is what happens. They build and set up, g...",NaN


In [42]:
words = []
pos_tags_dataset = []
for sentence in tqdm(dataset.iloc[:,2].values.tolist()):
    doc = pos_tagger(sentence)
    pos_tags_dataset.append([word.xpos for sent in doc.sentences for word in sent.words])
    words.append([word.text for sent in doc.sentences for word in sent.words])

 96%|█████████▋| 63374/65694 [2:52:10<06:18,  6.13it/s]  


AssertionError: input should be either str, list or Document

In [51]:
#Drop all rows with empty cleamed comments
dataset = dataset[dataset['cleaned_comment'].notna()]


In [55]:
import re
labels= []
word_indices=[]
for i,sentence in enumerate(dataset['cleaned_comment']):
    #capture bw indices for each sentence
    try:
        for j,word in enumerate(sentence.split()):
            if word.lower() in bad_words_set:
                pos_tags_dataset[i][j] = 'BW'
    except:
        continue

In [56]:
dataset['pos_tag'] = pos_tags_dataset

/tmp/ipykernel_71509/3001157243.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['pos_tag'] = pos_tags_dataset


In [58]:
dataset.to_csv('data/pos_tags_dataset_bw.csv',index=False)